In [1]:
%load_ext autoreload 
%autoreload 2
%reload_ext autoreload

import xarray as xr
import numpy as np
import os
import pandas as pd
from glob import glob
import datetime as dt
from PIL import Image
import sys
import functions as f
import pickle
import tensorflow as tf
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras import backend as K
from tensorflow.keras.utils import plot_model
from IPython.display import Image 
from tensorflow.python.data.ops import dataset_ops
import dask
from keras.models import load_model
from sklearn.metrics import mean_squared_error
from io import StringIO
import shap
from keras.layers import Input
import keras as k
from contextlib import redirect_stdout
from PIL import Image
import matplotlib.gridspec as gridspec
import cv2
import channelExperiment as CE
import loadDataWeek0 as loadData
import bottleneck as bn
from tensorflow.keras import mixed_precision
import csv
import addPredictors as pred
import loadValues as lv
import masks
import denseValue
import verifications
import preprocessUtils as putils
import matplotlib.pyplot as plt
import losses
import modelRzsmReluExtraConvResidual as UNETRzsm

global region_name,testing_scenario,save_loss_name
region_name = 'CONUS' #['australia','CONUS', 'china']

'''Make sure to fill in this information depending on what test we are doing'''
make_single_prediction_from_model_for_testing=False
make_additional_predictions_from_model_for_testing = False
permutation_test_graphs_create=True
bias_correction_predict = False

#Tensorflow RT things
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)
tf.config.optimizer.set_jit(True) # Enable XLA.


global RZSM_or_Tmax_or_both
RZSM_or_Tmax_or_both = 'RZSM' # for getting the predictor from either RZSM and Tmax ('both') or only RZSM ('RZSM')

global num_predictions_testing
num_predictions_testing = 25

global num_train_val_predictions
num_train_val_predictions=10

#for permutation plot
max_RZSM_value,max_tmax_value = 0.05,5

#set for the larger memeory ones
# os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'


global ref_source,final_testing_year
ref_source = 'GEFSv12'
final_testing_year = 2019

testing_scenario = 'regularResidual' #['dense', 'regular', 'transformer', 'super_pixel', 'attention', 'denseLarge', 'basic', 'regularResidual']

save_loss_name = 'regularResidual'
loss_fn = losses.crps2d_tf

2024-05-02 12:46:47.837949: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-02 12:46:48.655369: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/glade/work/klesinger/conda-envs/tf212gpu_new/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using TensorFlow backend


2024-05-02 12:46:53.598027: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30315 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:61:00.0, compute capability: 7.0


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: Tesla V100-SXM2-32GB, compute capability 7.0


# Choose different loss and architecture configurations

# This script will load data based on specific lead times that we want to experiment with and what Experiments we choose to run

# Week 1 (lead index 6 of the forecast)

| Ex. Number/Name | Obs. RZSM Inputs | Obs. Other Inputs                    | UNET predicted Inputs (N>=2)     |Reforecast Inputs| Prediction Lead Week/Variables | Activation function | Loss function | Batch size |
| ----------------| -----------------| ------------------------------------ | -------------------      |--------          | -------------------| ---------------------| ------------------------| ------------|
| 0 - EX0         | None             | None                                 | None                     | RZSM, Wk 1-N| Wk N     -  RZSM |              Relu             | CRPS experimental| 66
| 1 - EX1         | Wk. lags 1-3    | pwat,spfh,tmax,z200,diff_temp lags 1-3 | RZSM Wk 1-N |        None |  Wk N     -  RZSM |             Relu             | CRPS experimental | 66
| 2 - EX2         | Wk. lags 1-6   | pwat,spfh,tmax,z200,diff_temp lags 1-3  | RZSM Wk 1-N          |  None |  Wk N     -  RZSM |       Relu             | CRPS experimental| 66
| 3 - EX3         | Wk. lags 1-9    | pwat,spfh,tmax,z200,diff_temp lags 1-3 | RZSM Wk 1-N         |  None |  Wk N     -  RZSM |       Relu             | CRPS experimental| 66
| 4 - EX4         | Wk. lags 1-12    | pwat,spfh,tmax,z200,diff_temp lags 1-3| RZSM Wk 1-N         |  None |  Wk N     -  RZSM |      Relu             | CRPS experimental| 66
| 5 - EX5         | Wk. lags 1-3   | None                                   | None                   | RZSM, week N | Wk N     -  RZSM  |         Relu             | CRPS experimental| 66
| 6 - EX6         | Wk. lags 1-6   | None                                     | None                   |RZSM, week N  | Wk N     -  RZSM  |        Relu             | CRPS experimental| 66
| 7 - EX7         | Wk. lags 1-9    | None                                | None                   |RZSM, week N  | Wk N     -  RZSM  |         Relu             | CRPS experimental |66
| 8 - EX8         | Wk. lags 1-12    | None                                  | None                   |RZSM, week N  | Wk N     -  RZSM |         Relu             | CRPS experimental | 66
| 9 - EX9         | Wk. lags 1-3    |pwat,spfh,tmax,z200,diff_temp lags 1-3  | RZSM Wk 1-N                  |RZSM  week N  | Wk N     -  RZSM  |         Relu             | CRPS experimental| 66
| 10 - EX10       | Wk. lags 1-6    |pwat,spfh,tmax,z200,diff_temp  lags 1-3 | RZSM Wk 1-N                  |RZSM  week N  |Wk N     -  RZSM   |         Relu             | CRPS experimental| 66
| 11 - EX11       | Wk. lags 1-9    |pwat,spfh,tmax,z200,diff_temp  lags 1-3 | RZSM Wk 1-N                  |RZSM  week N  | Wk N     -  RZSM  |         Relu             | CRPS experimental| 66
| 12 - EX12       | Wk. lags 1-12   |pwat,spfh,tmax,z200,diff_temp lags 1-3  | RZSM Wk 1-N                  |RZSM  week N  | Wk N     -  RZSM  |         Relu             | CRPS experimental| 66
| 13 - EX13       | None             | None                                 | None                     | RZSM week N| Wk N     -  RZSM |              Relu             | CRPS experimental| 66
| 14 - EX14         | Wk. lags 1-3    | pwat,spfh,tmax,z200,diff_temp lags 1-3 | None |        None |  Wk N     -  RZSM |             Relu             | CRPS experimental | 66
| 15 - EX15         | Wk. lags 1-6   | pwat,spfh,tmax,z200,diff_temp lags 1-3  | None             |  None |  Wk N     -  RZSM |       Relu             | CRPS experimental| 66
| 16 - EX16         | Wk. lags 1-9    | pwat,spfh,tmax,z200,diff_temp lags 1-3 | None            |  None|  Wk N     -  RZSM |       Relu             | CRPS experimental| 66
| 17 - EX17        | Wk. lags 1-12    | pwat,spfh,tmax,z200,diff_temp lags 1-3| None            |  None|  Wk N     -  RZSM |      Relu             | CRPS experimental| 66
| 18 - EX18         | Wk. lags 1-3   | None                                   | RZSM Wk 1-N                 | RZSM, week N | Wk N     -  RZSM  |         Relu             | CRPS experimental| 66
| 19 - EX19         | Wk. lags 1-6   | None                                     | RZSM Wk 1-N                  |RZSM, week N  | Wk N     -  RZSM  |        Relu             | CRPS experimental| 66
| 20 - EX20         | Wk. lags 1-9    | None                                | RZSM Wk 1-N                  |RZSM, week N  | Wk N     -  RZSM  |         Relu             | CRPS experimental |66
| 21 - EX21         | Wk. lags 1-12    | None                                  | RZSM Wk 1-N                   |RZSM, week N  | Wk N     -  RZSM |         Relu             | CRPS experimental | 66
| 22 - EX22         | Wk. lags 1-3   | None                                   | None                   | None | Wk N     -  RZSM  |         Relu             | CRPS experimental| 66
| 23 - EX23         | Wk. lags 1-6   | None                                     | None                   |None  | Wk N     -  RZSM  |        Relu             | CRPS experimental| 66
| 24 - EX24         | Wk. lags 1-9    | None                                | None                   |None  | Wk N     -  RZSM  |         Relu             | CRPS experimental |66
| 25 - EX25         | Wk. lags 1-12    | None                                  | None                   |None  | Wk N     -  RZSM |         Relu             | CRPS experimental | 66
| 26 - EX26         | None    | None                                  | None                   |Wk 1-N (choose best models)  | Wk N     -  RZSM |         Relu             | CRPS experimental | 66
| 27 - EX27         | Wk. lags 1-3    |pwat,spfh,tmax,z200,diff_temp lags 1-3  |  None                   |Wk1 RZSM,tmax, diff_temp, z200, pwat, spfh  | Wk N     -  RZSM  |         Relu             | CRPS experimental| 66 (like EX9)
| 28 - EX28         | Wk. lags 1-6    |pwat,spfh,tmax,z200,diff_temp lags 1-3  |  None                   |Wk1 RZSM,tmax, diff_temp, z200, pwat, spfh  | Wk N     -  RZSM  |         Relu             | CRPS experimental| 66 (like EX10)

# EX 1-4 - Testing if the prediction of previous week adds value (but don't include the current week that is trying to be predicted). Observation driven but with a prediction.
# EX 14-17 - Testing if the prediction of previous week adds value (but don't include the current week that is trying to be predicted). Purely observation driven.
# EX 5-8 - Testing if adding other observations (pwat, spfh, etc) has an increase or decrease in skill.
# EX 18-21 - Seeing if adding the previous week gains additional skill
# EX 22-25 - Purely observation driven. Only soil moisture.

In [2]:
input_directory = f'Data/model_npy_inputs/{region_name}/Model_input_data' #For inputs into the model
verification_directory = f'Data/model_npy_inputs/{region_name}/Verification_data' #For observation verification
start_of_forecast_init = '2000-01-05'
end_of_forecast_init = '2019-12-25'

global training_size_shape
training_size_shape = np.array((9185,48,96))

global validation_testing_size_shape
validation_testing_size_shape = np.array((1144,48,96))

'''This decides how many lag weeks we have for data such as pwat, z200, tmax, etc'''
global observation_lag_list_not_RZSM
#This is for observations pwat, z200, spfh, tmax, diff_temp variables used as predictors
#These are the day lags which were already computed as the 7-day rolling mean

observation_lag_list_not_RZSM = [-1,-7,-14] 

global train_start, train_end, val_start, val_end, test_start, test_end
train_start = '2000-01-01'
train_end = '2015-12-31'

val_start = '2016-01-01'
val_end = '2017-12-31'

test_start = '2018-01-01'
test_end = '2019-12-31'


# Only testing EX9 first

In [3]:
def load_all_data_EX9_EX10_EX11_EX12_after_week_0(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, include_reforecast_or_not, 
                                           observation_lag_list_not_RZSM, lag_integer_list, input_directory,training_size_shape,
                                           validation_testing_size_shape,experiment_name,RZSM_or_Tmax_or_both, experiment_test,region_name,experiment_name_out):
    '''We need to combine all the RZSM files and all the observation data (pwat, spfh, tmax, diff_temp, z200) into one file. Also adding the RZSM reforecast data from RZSM and Tmax '''

    channel_list = []
    
    #Reforecast predictors
    var_list = ['pwat_eatm', 'spfh_2m', 'tmax_2m', 'diff_temp_2m', 'hgt_pres']

    if experiment_test == 0:
        add_channels = lead  
        reforecast_predictors = False
    elif (experiment_test >= 1) and (lead in [1,2]):
        #This is adding 5 additional predictors (tmax, diff_temp, z200, pwat, spfh)
        #Also add the number of channels according to the lead. If lead == 1, then only add lead 1 current week. If lead == 2, add one channel for week 2 and one channel for the prediction from Week 1
        add_channels = lead  + len(var_list)
        reforecast_predictors = True
    elif (experiment_test >= 1) and (lead > 2):
        add_channels = lead 
        reforecast_predictors = False
            
   
    training_input = np.empty(shape = (training_size_shape[0],training_size_shape[1],training_size_shape[2],len(lag_integer_list)+len(var_list)*3+add_channels))
    validation_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],len(lag_integer_list)+len(var_list)*3+add_channels))
    testing_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],len(lag_integer_list)+len(var_list)*3+add_channels))
    
    print(f'Training input shape = {training_input.shape}')
    
    #Add RZSM observations first
    for idx,lag in enumerate(lag_integer_list):
        channel_list.append(f'RZSM_obs_lag{lag}')
        training_input, validation_input, testing_input = pred.add_obs_RZSM_by_lag(training_input, validation_input, testing_input, lag, input_directory, idx, final_testing_year)
        
    print(f'Index idx value is {idx}. Done adding RZSM obs.')    
    
    RZSM_train_obs, RZSM_validation_obs = pred.return_masking_objects_for_RZSM(input_directory, final_testing_year)

    for variable in var_list:
        channel_name_output = pred.return_channel_name(variable)
        for lag in observation_lag_list_not_RZSM:
            channel_list.append(f'{channel_name_output}_obs_lag{lag}')
            idx+=1
            #Observations adding
            training_input, validation_input, testing_input = pred.add_obs_other_observations_by_lag(training_input, validation_input, testing_input, lag, input_directory, variable, idx,final_testing_year)
 
    
        print(f'Index idx value is {idx}. Done adding {variable} obs.')
    
    ############################### NOW ADD THE PREDICTION DATA FROM PREVIOUS WEEK ################################################
    #Now we need to load our data from the previous weeks 
    
    if lead in [0, 1]:
        #DO NOT INCLUDE LEAD 0 AS A PREDICTOR (FROM PREVIOUS WEEK)
        pass
    else:

        for lead_previous in range(1,lead):
            # break
            print(f'Adding previous RZSM prediction from week {lead_previous} as an input channel')
            # break
            
            pred_dir = f'predictions/{region_name}/Wk{lead_previous}_training_validation'
            testing_dir = f'predictions/{region_name}/Wk{lead_previous}_testing'
            
            os.system(f'mkdir -p {pred_dir}')
            
            training_prediction_name= f'{pred_dir}/Wk{lead_previous}_training_{experiment_name_out}.npy'
            validation_prediction_name = f'{pred_dir}/Wk{lead_previous}_validation_{experiment_name_out}.npy'
            testing_prediction_name = f'{testing_dir}/Wk{lead_previous}_testing_{experiment_name_out}.npy'
            
            testing_prediction = np.load(testing_prediction_name)
            
            '''Load the training and validation predictions if they exist'''
            if os.path.exists(training_prediction_name):
                print(f'Loading data from previous week lead {lead_previous}')
                training_prediction = np.load(training_prediction_name)
                validation_prediction =np.load(validation_prediction_name)
                
            else:
                print(f'Creating prediction data from previous week lead {lead_previous}')
                '''If they don't exist..... load model for previous week'''
                model = load_model(f'checkpoints/Wk_{lead_previous}/Wk{lead_previous}_{experiment_name_out}',compile=False) #don't need the custom loss function for predictions
                
                training_input_previous = np.load(f'Data/model_npy_inputs/{region_name}/Wk{lead_previous}_EX_input_data/{experiment_name_out}_training_input.npy')
                validation_input_previous = np.load(f'Data/model_npy_inputs/{region_name}/Wk{lead_previous}_EX_input_data/{experiment_name_out}_validation_input.npy')
                testing_input_previous = np.load(f'Data/model_npy_inputs/{region_name}/Wk{lead_previous}_EX_input_data/{experiment_name_out}_testing_input.npy')
                
                '''Make predictions and save data to load for previous use'''
                training_prediction = model.predict(training_input_previous) #Just for unit checking
                np.save(training_prediction_name,training_prediction)
                
                validation_prediction = model.predict(validation_input_previous)
                np.save(validation_prediction_name,validation_prediction)

            train_RZSM = training_prediction
            val_RZSM = validation_prediction
            test_RZSM = testing_prediction

            #Now add back to the data
            '''Now add back to the newly created dataset'''
            idx+=1
            channel_list.append(f'RZSM_prediction_lead{lead_previous}')
            print(f'Adding RZSM training, validation, testing into index {idx}')
            training_input[:,:,:,idx] = train_RZSM[:,:,:,0]
            validation_input[:,:,:,idx] = val_RZSM[:,:,:,0]
            testing_input[:,:,:,idx] = test_RZSM[:,:,:,0]


    #Add final predictor for the current week of RZSM within reforecast
    idx+=1
    soil_var = 'soilw_bgrnd'
    channel_name_output = pred.return_channel_name(soil_var)
    channel_list.append(f'{channel_name_output}_ref_lead{lead}')

    print(f'And finally adding RZSM week {lead} from reforecast as a predictor')
    
    training_input, validation_input, testing_input = pred.add_reforecast_by_lag(training_input, validation_input, testing_input, lead, input_directory,soil_var, idx, ref_source, final_testing_year)

    print(f'Index idx value is {idx}. Done adding {soil_var} reforecast.')

    print('Final channel of input is the following (just make sure it has zeros for RZSM)')
    print(training_input[0,:,:,-1])

    return(training_input, validation_input, testing_input,channel_list)




In [4]:
def make_UNET_stacked_inputs(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, include_reforecast_or_not, 
                             observation_lag_list_not_RZSM, lag_integer_list, input_directory,training_size_shape,
                             validation_testing_size_shape,experiment_name,RZSM_or_Tmax_or_both,addtl_experiment,experiment_test,region_name,experiment_name_out):
    

    
    ############################################### EXPERIMENTS 9-12 ####################################################################################
    if (experiment_name == 'EX9') or (experiment_name == 'EX10') or (experiment_name == 'EX11') or (experiment_name == 'EX12') or (experiment_name == 'EX27') or (experiment_name == 'EX28'):
        
        print(f'\nWorking on setting up data for Experiment {experiment_name} for lead {lead}\n')

        if lead == 0:
            final_input_train, final_input_validation, final_input_testing,channel_list = loadData.load_all_data_EX9_EX10_EX11_EX12(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, 
                                                                                                              include_reforecast_or_not, observation_lag_list_not_RZSM, lag_integer_list, 
                                                                                                              input_directory,training_size_shape,validation_testing_size_shape,RZSM_or_Tmax_or_both, 
                                                                                                                                    experiment_test = experiment_test,
                                                                                                                                   region_name=region_name)
        else:
            final_input_train, final_input_validation, final_input_testing,channel_list = load_all_data_EX9_EX10_EX11_EX12_after_week_0(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, include_reforecast_or_not, 
                                                                                                       observation_lag_list_not_RZSM, lag_integer_list, input_directory,training_size_shape,
                                                                                                   validation_testing_size_shape,experiment_name,RZSM_or_Tmax_or_both,
                                                                                                                                       experiment_test = experiment_test,
                                                                                                                                       region_name = region_name,experiment_name_out=experiment_name_out)

    return (final_input_train,final_input_validation,final_input_testing,channel_list)


In [5]:
#To try and save on memory, only load certain files at a time
def return_only_train_validation(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, include_reforecast_or_not, 
                                 observation_lag_list_not_RZSM, lag_integer_list, input_directory,training_size_shape,
                                 validation_testing_size_shape,experiment_name):
    
    reforecast_train_input, reforecast_validation_input, reforecast_testing_input,channel_list \
    = make_UNET_stacked_inputs(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, 
                               include_reforecast_or_not, observation_lag_list_not_RZSM, 
                               lag_integer_list, input_directory,training_size_shape,validation_testing_size_shape,experiment_name,region_name)

    return(reforecast_train_input, reforecast_validation_input,reforecast_testing_input,channel_list)





def return_testing_data_model_training(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, 
                               include_reforecast_or_not, observation_lag_list_not_RZSM, 
                               lag_integer_list, input_directory,training_size_shape,validation_testing_size_shape,experiment_name):

    reforecast_train_input, reforecast_validation_input, reforecast_testing_input,channel_list = make_UNET_stacked_inputs(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, 
                               include_reforecast_or_not, observation_lag_list_not_RZSM, 
                               lag_integer_list, input_directory,training_size_shape,validation_testing_size_shape,experiment_name,region_name)
    return(reforecast_testing_input,channel_list)

# Load Data files

In [6]:
# ## Testing
# region_name=region_name
# lead = lead_week
# num_lags_obs_RZSM=3
# include_lags_obs_pwat_spfh_tmax=True
# include_reforecast_or_not=True
# addtl_experiment = False
# experiment_test = 1

In [7]:
def return_experiment_name_and_lags_and_channel_dir(include_reforecast_or_not, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, addtl_experiment, experiment_test, region_name):
    
    experiment_name = CE.return_experiment_name(include_reforecast_or_not,num_lags_obs_RZSM,include_lags_obs_pwat_spfh_tmax,addtl_experiment,experiment_test)

    save_experiment_name = f'{experiment_name}_{save_loss_name}_RZSM'
    experiment_name_out =  save_experiment_name
    
    global lag_integer_list
    lag_integer_list =  CE.return_num_day_lags_from_weekly_lags(num_lags_obs_RZSM) #For number of RZSM observation lags

    #Where to save channel information
    save_experiment_dir = f'Data/model_npy_inputs/{region_name}/Wk{lead}_EX_input_data'
    
    channel_save_dir = f'channel_list_information/Wk{lead}'
    
    checkpoint_filepath = f'checkpoints/{region_name}/Wk{lead}/Wk{lead}_{experiment_name_out}'
    
    losses_dir = f'Losses_with_OBS/{region_name}/Wk{lead}'
    
    save_checkpoint_dir =f'checkpoints/{region_name}/Wk{lead}'
    
    os.system(f'mkdir -p {checkpoint_filepath} {channel_save_dir} {save_experiment_dir} {losses_dir} {save_checkpoint_dir}')

    
    #Set up files for either saving or loading
    training_input_file = f'{save_experiment_dir}/{save_experiment_name}_training_input.npy'
    validation_input_file = f'{save_experiment_dir}/{save_experiment_name}_validation_input.npy'
    testing_input_file = f'{save_experiment_dir}/{save_experiment_name}_testing_input.npy'

    
    return(save_experiment_name,experiment_name_out,experiment_name,lag_integer_list,channel_save_dir,
           training_input_file,validation_input_file,testing_input_file,checkpoint_filepath,
          losses_dir,save_checkpoint_dir,save_experiment_dir)

In [8]:
def generate_model_inputs(region_name,lead,include_reforecast_or_not,num_lags_obs_RZSM,include_lags_obs_pwat_spfh_tmax,addtl_experiment,experiment_test):

    save_experiment_name, experiment_name_out, experiment_name,lag_integer_list, \
    channel_save_dir, training_input_file, validation_input_file, \
    testing_input_file,checkpoint_filepath, losses_dir,save_checkpoint_dir, \
    save_experiment_dir = return_experiment_name_and_lags_and_channel_dir(include_reforecast_or_not, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, addtl_experiment, experiment_test, region_name)

    if os.path.exists(training_input_file) and os.path.exists(validation_input_file) and os.path.exists(testing_input_file):
        pass
    else:
        reforecast_train_input, reforecast_validation_input, reforecast_testing_input, channel_list \
        = make_UNET_stacked_inputs(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, include_reforecast_or_not, 
                             observation_lag_list_not_RZSM, lag_integer_list, input_directory,training_size_shape,
                             validation_testing_size_shape,experiment_name,RZSM_or_Tmax_or_both,addtl_experiment,experiment_test,region_name,experiment_name_out)
    
        #Save data to file
        np.save(training_input_file,tf.convert_to_tensor(reforecast_train_input,dtype=tf.float32))
        np.save(validation_input_file,tf.convert_to_tensor(reforecast_validation_input,dtype=tf.float32))
        np.save(testing_input_file,tf.convert_to_tensor(reforecast_testing_input,dtype=tf.float32))
        
        image_size = reforecast_train_input.shape[1:]           
        #Save channel list information to txt file
    
        with open(f'{channel_save_dir}/{save_experiment_name}_channel_list.txt', 'w') as file:
            for idx,element in enumerate(channel_list):
                if '-' in element:
                    source_='OBSERVATIONS'
                else:
                    source_='REFORECAST'
    
                file.write(f'Channel_{idx} is from {source_} with lead or lag {str(element)}' + '\n')

    return(f'Completed writing lead {lead} model input data to {save_experiment_dir}.')

In [9]:
def train(epochs, batch_size, lead, initial_learning_rate, beta_1,shuffle,patience, kernel_norm,deep_supervision,
         num_lags_obs_RZSM,include_lags_obs_pwat_spfh_tmax,include_reforecast_or_not,
          number_of_UNET_backbone_max_pool,permutation_test,make_additional_predictions_from_model_for_testing,
         input_directory,training_size_shape,validation_testing_size_shape,addtl_experiment,experiment_test,region_name):
    
#For testing
# lead=1
# num_lags_obs_RZSM = 3
# include_lags_obs_pwat_spfh_tmax = True
# include_reforecast_or_not=True
# deep_supervision = True
# initial_learning_rate = 0.0001
# beta_1 = 0.9
# batch_size=66
# epochs=1
# shuffle=False
# kernel_norm =  None
# patience=10
# permutation_test = False
    
    # print(addtl_experiment)
    #Training data
    #Needed for calculating the loss for individual UNET predictions (each has 4 predictions for each variable)
    def return_training_verification_data(RZSM_or_tmax):
        #We had multiple verification inputs in the past, but now we just reduced to RZSM
        if RZSM_or_tmax == 'RZSM':
            return(np.array(tf.expand_dims(obs_final_train,-1)))

    
    #Validation data
    def return_validation_verification_data(RZSM_or_tmax):
        #We had multiple verification inputs in the past, but now we just reduced to RZSM
        if RZSM_or_tmax == 'RZSM':
            return(np.array(tf.expand_dims(obs_final_validation,-1)))

    
    
    def print_shape(file,name):
        print(f'Shape of {name} is {file.shape}')
        
    save_experiment_name, experiment_name_out, experiment_name,lag_integer_list, \
    channel_save_dir, training_input_file, validation_input_file, \
    testing_input_file,checkpoint_filepath, losses_dir,save_checkpoint_dir, \
    save_experiment_dir = return_experiment_name_and_lags_and_channel_dir(include_reforecast_or_not, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, addtl_experiment, experiment_test, region_name)

    obs_final_train,obs_final_validation,obs_final_testing = f.load_verification_observations_residuals(lead,verification_directory)
    

    print('Loading previously created data')

    reforecast_train_input_tensor = tf.data.Dataset.from_tensor_slices(({'input_image':np.load(training_input_file)},
                                                                        {'RZSM_output_1': return_training_verification_data('RZSM'), 'RZSM_output_2': return_training_verification_data('RZSM'), }))
    reforecast_validation_input_tensor = tf.data.Dataset.from_tensor_slices(({'input_image':np.load(validation_input_file)},
                                                                        {'RZSM_output_1': return_validation_verification_data('RZSM'), 'RZSM_output_2': return_validation_verification_data('RZSM'),}))

    image_size = np.array(list(reforecast_train_input_tensor.element_spec)[0]['input_image'].shape)
 

    with open(f'{channel_save_dir}/{save_experiment_name}_channel_list.txt', 'r') as file:
        # Read all lines from the file into a list
        channel_list = file.readlines()

    # print_shape(reforecast_train_input,'Training input')
    # print_shape(reforecast_validation_input,'Validation input')
    # print_shape(reforecast_testing_input,'Testing input')
    # print_shape(obs_final_train,'Observation verification training')
    # print_shape(obs_final_validation,'Observation verification validation')
    # print_shape(obs_final_testing,'Observation verification testing')
    

    print('Actual channel order list')
    print(channel_list)

    inputs = Input(shape=image_size, name='input_image')
    
    print('\nCompiling model\n')

    build_model = UNETRzsm.model_build_func(inputs=inputs, output_channels=1, 
                           using_deep_supervision=deep_supervision, 
                           kernel_norm = kernel_norm , var_name='RZSM',
                          number_of_UNET_backbone_max_pool = number_of_UNET_backbone_max_pool)
    model = Model(inputs=inputs,
             outputs = build_model,
             name="UNET_RZSM")


    model.compile(loss= {'RZSM_output_1':loss_fn, 'RZSM_output_2':loss_fn},
              metrics = 'mae', optimizer = k.optimizers.Adam(learning_rate=initial_learning_rate, beta_1=beta_1),run_eagerly=True)

    '''For some reason I can't save anymore to my home space, so let's just move it to scratch'''

    checkpoint_filepath_out = f'/glade/derecho/scratch/klesinger/{checkpoint_filepath}'
    save_checkpoint_dir_out = f'/glade/derecho/scratch/klesinger/{save_checkpoint_dir}'

    os.system(f'rm {checkpoint_filepath} -r') #must do this so that we can make a soft link
    os.system(f'mkdir -p {save_checkpoint_dir_out} {save_checkpoint_dir}')
    
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath_out,
    save_weights_only=True,
    mode='min',
    save_best_only=True)
    
    rlrop = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)
    
    my_callbacks = [tf.keras.callbacks.EarlyStopping(patience=patience),model_checkpoint_callback, rlrop] 
    
    unet_history=model.fit(
        x=reforecast_train_input_tensor.batch(batch_size), 
        batch_size=batch_size, epochs=epochs, initial_epoch=0, shuffle=shuffle, callbacks=my_callbacks, \
        validation_data=reforecast_validation_input_tensor.batch(batch_size))

    loss_out_df = pd.DataFrame(unet_history.history)
    loss_out_df.to_csv(f'{losses_dir}/Wk{lead}_{experiment_name_out}')


    ########### MODEL CHECKPOINTS ############################

    model.save(f'/glade/work/klesinger/FD_RZSM_deep_learning/{save_checkpoint_dir}/Wk{lead}_{experiment_name_out}')

    #If saving to a scratch location
    # model.save(f'{save_checkpoint_dir_out}/Wk{lead}_{experiment_name_out}')
    # os.system(f'ln -s {save_checkpoint_dir_out}/Wk{lead}_{experiment_name_out} /glade/work/klesinger/FD_RZSM_deep_learning/{save_checkpoint_dir}/Wk{lead}_{experiment_name_out}')
              
    return(0)


    

In [10]:
# # #Testing for model runs
# permutation_test=False
# lead=1
# num_lags_obs_RZSM =3
# include_lags_obs_pwat_spfh_tmax = True
# include_reforecast_or_not=True
# deep_supervision = True
# initial_learning_rate = 0.0001
# beta_1 = 0.9
# batch_size=66
# epochs=1
# shuffle=False
# kernel_norm =  None
# patience=10
# number_of_UNET_backbone_max_pool=4
# make_additional_predictions_from_model_for_testing=False
# addtl_experiment = False

# Run the model for each experiment

In [11]:
'''For experiments, you need to run the script once to make the data. Then restart the kernel to load the data. 
Otherwise the memory leaks are too high'''
def run_EXPERIMENT(lead, num_lags_obs_RZSM,include_lags_obs_pwat_spfh_tmax ,include_reforecast_or_not,addtl_experiment,experiment_test,region_name):
    if lead_week == 5:
        patience=15
    else:
        patience = 10

    #Options for num_lags_obs_RZSM weekly lags [ 3,6,9,12 ] ** this inc
    #Options for num_lags_obs_pwat_spfh_tmax weekly lags [0 weeks or 2 weeks ]

    train(epochs = 300, 
          batch_size = 66, 
          lead=lead_week, 
          initial_learning_rate=0.0001,
          beta_1 = 0.9,
          shuffle=False,
          patience=patience,
          kernel_norm = None,
          deep_supervision = True,
          num_lags_obs_RZSM=num_lags_obs_RZSM, 
          include_lags_obs_pwat_spfh_tmax=include_lags_obs_pwat_spfh_tmax,
          include_reforecast_or_not=include_reforecast_or_not,
          number_of_UNET_backbone_max_pool=4,
          permutation_test = False,
          make_additional_predictions_from_model_for_testing = False,
          input_directory=input_directory,
          training_size_shape=training_size_shape,
          validation_testing_size_shape=validation_testing_size_shape,
          addtl_experiment = addtl_experiment,
          experiment_test =experiment_test,
          region_name = region_name)
    return(0)



In [12]:
def reverse_min_max_scaling(file,region_name,lead,final_testing_year):
    if final_testing_year == 2012:
        add_ = f"_{test_year}"
    elif final_testing_year == 2019:
        add_ = ""

    source = 'GEFSv12'
    max_min_source = f'Data/min_max_values/{region_name}'
    
    max_, min_ = np.load(f'{max_min_source}/residual_{source}_lead{lead}_max{add_}.npy'), np.load(f'{max_min_source}/residual_{source}_lead{lead}_min{add_}.npy')
    return(file *(max_-min_)+min_)

In [42]:
def make_single_predictions_testing(lead, include_reforecast_or_not, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax,addtl_experiment,experiment_test, region_name, test):
    
    save_experiment_name, experiment_name_out, experiment_name,lag_integer_list, \
    channel_save_dir, training_input_file, validation_input_file, \
    testing_input_file,checkpoint_filepath, losses_dir,save_checkpoint_dir, \
    save_experiment_dir = return_experiment_name_and_lags_and_channel_dir(include_reforecast_or_not, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, addtl_experiment, experiment_test, region_name)

    print(f'Testing input file is: {testing_input_file}')
    ########### MAKE PREDICTIONS ON TRAINING, TESTING, VALIDATION SET ############################
    # model = load_model(f'checkpoints/Wk_{lead}/Wk{lead}_{experiment_name_out}',compile=False)

    print(f'Starting single prediction for Experiment {experiment_name_out}')

    day_num = (lead*7) - 1

    
    try:
        model = load_model(checkpoint_filepath,compile=False) #don't need the custom loss function for predictions
        
        #Load data
        reforecast_testing_input = np.load(testing_input_file)
        
        ########### SAVE PREDICTIONS (SINGLE PREDICTIONS) ############################
        test_dir = f'predictions/{region_name}/Wk{lead}_testing'
    
        os.system(f'mkdir -p {test_dir}')
    
        mask = masks.load_mask(region_name)
        mask = mask.transpose('Y','X','S')
    
        print('\nCurrently only predicting the test dataset\n')
        predictions = model.predict(reforecast_testing_input)

        #Only grab the last channel
        predictions = np.array(predictions)[-1,:,:,:,:] #this is in min max format
        
        
        #Convert back prior to min max
        residuals = reverse_min_max_scaling(predictions,region_name,day_num,final_testing_year)
        residuals = np.where(mask['NCA-LDAS_mask'] == 1,residuals,0)
        # residuals.max()
        # residuals.min()
        
        #Convert original forecasts back prior to min max. This is the anomaly of the observation.
        original = reverse_min_max_scaling(np.expand_dims(test,-1),region_name,day_num,final_testing_year)
        original = np.where(mask['NCA-LDAS_mask'] == 1,original,0)
    
        '''Add residuals to original datasets min max'''
        test_out = residuals - original
        test_out = np.where(original == 0, 0, test_out)
        # predictions.shape
        np.save(f'{test_dir}/Wk{lead}_testing_{experiment_name_out}.npy',test_out)
    except NameError:
        print('Error loading model')
        pass
    except OSError:
        print('Error loading model')
        pass
    
    return(f'Completed Experiment {experiment_name_out}')

In [14]:
def make_single_predictions_training_validation(lead, include_reforecast_or_not, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax,addtl_experiment,experiment_test, region_name, train, val):
    
    save_experiment_name, experiment_name_out, experiment_name,lag_integer_list, \
    channel_save_dir, training_input_file, validation_input_file, \
    testing_input_file,checkpoint_filepath, losses_dir,save_checkpoint_dir, \
    save_experiment_dir = return_experiment_name_and_lags_and_channel_dir(include_reforecast_or_not, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, addtl_experiment, experiment_test, region_name)

    ########### MAKE PREDICTIONS ON TRAINING, TESTING, VALIDATION SET ############################
    # model = load_model(f'checkpoints/Wk_{lead}/Wk{lead}_{experiment_name_out}',compile=False)

    print(f'Starting single prediction for Experiment {experiment_name_out}')
       
    model = load_model(checkpoint_filepath,compile=False) #don't need the custom loss function for predictions
    
    #Load data
    reforecast_training_input = np.load(training_input_file)
    reforecast_validation_input = np.load(validation_input_file)

    day_num = (lead*7) - 1
    
    ########### SAVE PREDICTIONS (SINGLE PREDICTIONS) ############################
    train_val_dir = f'predictions/{region_name}/Wk{lead}_training_validation'

    os.system(f'mkdir -p {train_val_dir}')

    mask = masks.load_mask(region_name)
    mask = mask.transpose('Y','X','S')

    print('\nPredicting training and validation input')
    predictions = model.predict(reforecast_training_input)
    #Only grab the last channel
    predictions = np.array(predictions)[-1,:,:,:,:]

    #Convert original forecasts back prior to min max
    original = reverse_min_max_scaling(np.expand_dims(train,-1),region_name,day_num,final_testing_year)
    original = np.where(mask['NCA-LDAS_mask'] == 1,original,0)

    '''Add residuals to original datasets min max'''
    train_out = predictions - original
    train_out = np.where(original == 0, 0, train_out)
    # predictions.shape
    np.save(f'{train_val_dir}/Wk{lead}_training_{experiment_name_out}.npy',train_out)

    #predict the validation data
    predictions = model.predict(reforecast_validation_input)
    
    #Only grab the last channel
    predictions = np.array(predictions)[-1,:,:,:,:]
    
    #Convert original forecasts back prior to min max
    original = reverse_min_max_scaling(np.expand_dims(val,-1),region_name,day_num,final_testing_year)
    original = np.where(mask['NCA-LDAS_mask'] == 1,original,0)

    '''Add residuals to original datasets min max'''
    val_out = predictions - original
    val_out = np.where(original == 0, 0, val_out)
    # predictions.shape
    np.save(f'{train_val_dir}/Wk{lead}_validation_{experiment_name_out}.npy',val_out)
       
    return(f'Completed Experiment {experiment_name_out}')

In [15]:
#These are models which only include the reforecast predictions. We are not including any observations into them.

EX0 ={'region_name':region_name, 'num_lags_obs_RZSM': 0, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':0 }
EX13={'region_name':region_name, 'num_lags_obs_RZSM': 0, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':1 }


In [16]:
#Include all predictors for obs but DO NOT include the current reforecast RZSM 

EX1 ={'region_name':region_name, 'num_lags_obs_RZSM': 3, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': False, 'addtl_experiment': False, 'experiment_test':0 }
EX2 ={'region_name':region_name, 'num_lags_obs_RZSM': 6, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': False, 'addtl_experiment': False, 'experiment_test':0 }
EX3 ={'region_name':region_name, 'num_lags_obs_RZSM': 9, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': False, 'addtl_experiment': False, 'experiment_test':0 }
EX4 ={'region_name':region_name, 'num_lags_obs_RZSM': 12, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': False, 'addtl_experiment': False, 'experiment_test':0 }


#Include all predictors for obs but DO include the current reforecast RZSM 
EX14 ={'region_name':region_name, 'num_lags_obs_RZSM': 3, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': False, 'addtl_experiment': True, 'experiment_test':1 }
EX15 ={'region_name':region_name, 'num_lags_obs_RZSM': 6, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': False, 'addtl_experiment': True, 'experiment_test':2 }
EX16 ={'region_name':region_name, 'num_lags_obs_RZSM': 9, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': False, 'addtl_experiment': False, 'experiment_test':3 }
EX17 ={'region_name':region_name, 'num_lags_obs_RZSM': 12, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': False, 'addtl_experiment': False, 'experiment_test':4 }


In [17]:
#Include only OBS RZSM predictors and the current reforecast RZSM week 

EX5 ={'region_name':region_name, 'num_lags_obs_RZSM': 3, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':0 }
EX6 ={'region_name':region_name, 'num_lags_obs_RZSM': 6, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':0 }
EX7 ={'region_name':region_name, 'num_lags_obs_RZSM': 9, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':0 }
EX8 ={'region_name':region_name, 'num_lags_obs_RZSM': 12, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':0 }

#Include only OBS RZSM predictors, previous weeks prediction, and current reforecast RZSM week 
EX18 ={'region_name':region_name, 'num_lags_obs_RZSM': 3, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': True, 'experiment_test':1 }
EX19 ={'region_name':region_name, 'num_lags_obs_RZSM': 6, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': True, 'experiment_test':2 }
EX20 ={'region_name':region_name, 'num_lags_obs_RZSM': 9, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': True, 'experiment_test':3 }
EX21 ={'region_name':region_name, 'num_lags_obs_RZSM': 12, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': True, 'experiment_test':4 }

#Include only OBS RZSM predictors DO NOT INCLUDE ANY REFORECAST OR PREDICTION
EX22 ={'region_name':region_name, 'num_lags_obs_RZSM': 3, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': True, 'experiment_test':5 }
EX23 ={'region_name':region_name, 'num_lags_obs_RZSM': 6, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': True, 'experiment_test':6 }
EX24 ={'region_name':region_name, 'num_lags_obs_RZSM': 9, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': True, 'experiment_test':7 }
EX25 ={'region_name':region_name, 'num_lags_obs_RZSM': 12, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': True, 'experiment_test':8 }


In [18]:
#Include only OBS RZSM predictors, OBS other variables, previous predictions of RZSM, and current reforecast lead

EX9 ={'region_name':region_name, 'num_lags_obs_RZSM': 3, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':0 }
EX10 ={'region_name':region_name, 'num_lags_obs_RZSM': 6, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':0 }
EX11 ={'region_name':region_name, 'num_lags_obs_RZSM': 9, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':0 }
EX12 ={'region_name':region_name, 'num_lags_obs_RZSM': 12, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':0 }


In [19]:

# EX26 ={'region_name':region_name, 'num_lags_obs_RZSM': 3, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':0 }

# run_EXPERIMENT(num_lags_obs_RZSM=0,include_lags_obs_pwat_spfh_tmax=False,include_reforecast_or_not=True, addtl_experiment = False, experiment_test = 26)

In [20]:
#This like EX9 and EX10 except now we are adding additional reforecast predictors from week 1 and 2

EX27 ={'region_name':region_name, 'num_lags_obs_RZSM': 3, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':1 }
EX28 ={'region_name':region_name, 'num_lags_obs_RZSM': 6, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':1 }



# Permutation test only

In [21]:
def setup_plot_permutation(lead,dictionary,metric):
    test=pd.DataFrame(dictionary,index=[metric])
    return(test)

In [22]:
def plot_barplot_mae_rmse(save_for_plot_mae, save_for_plot_rmse, lead, model_name):
    save_permutation_figures = f'Outputs/permutation_tests/barplots/{region_name}/Wk{lead}'
    os.system(f'mkdir -p {save_permutation_figures}')
    
    mae = setup_plot_permutation(lead,save_for_plot_mae,'MAE')
    rmse = setup_plot_permutation(lead,save_for_plot_rmse,'RMSE')
       
    # Plotting
    fig, axs = plt.subplots(2)
    
    # Plot for DataFrame 1
    mae.plot(kind='bar', ax=axs[0])
    axs[0].set_title('MAE')
    
    # Plot for DataFrame 2
    rmse.plot(kind='bar', ax=axs[1])
    axs[1].set_title('RMSE')
    
    # Adjust layout
    # plt.tight_layout()
    plt.savefig(f'{save_permutation_figures}/{model_name}.png')
    plt.show()

    return(0)
    

In [23]:
def create_plot_permutation(lead,max_RZSM_value,region_name):
    save_permutation_figures = f'Outputs/permutation_tests/{region_name}/plots/Wk{lead}'
    os.system(f'mkdir -p {save_permutation_figures}')

    
    # del plot_['Lead']
    plot_ = plot_.T #Must transpose to make it plot properly

    fig = plt.figure(figsize=(8,6))

    ax = fig.add_subplot(111) # Create matplotlib axes

    plot_.RZSM.plot(kind='bar', color='red',width=0.3, ax=ax, position=1)

    ax.set_ylabel('RZSM anomaly MAE')
    ax.set_ylim(0, max_RZSM_value)

    # plot_.plot(kind='bar')
    # plt.ylabel(f'{error_}')
    plt.suptitle(f'Permutation Test\nObservation values are in legend\nWk{lead} {experiment_name_out}',fontsize=10)
    
    legend1 = ax.legend(loc='upper right', bbox_to_anchor=(1.0, 1))

    ax.set_xlabel('\nPermutated Channels',weight='bold')
    plt.tight_layout()

    plt.savefig(f'{save_permutation_figures}/{experiment_name_out}.png')
    #plt.savefig(f'{save_permutation_figures}/{experiment_name}.tiff', format='tiff', dpi=300)
    #tiff.imsave(f'{save_permutation_figures}/{experiment_name}.tiff', data_array)
    plt.show()

    return(0)

In [24]:
def return_data_and_experiment_numbers(region_name, leads,train_start, train_end, val_start, val_end, test_start, test_end,day_num):
        
    obs_final_train,obs_final_validation,obs_final_testing = verifications.open_obs_for_verification(region_name, leads,train_start, train_end, val_start, val_end, test_start, test_end)

    obs_final_testing = obs_final_testing.sel(L=day_num)
    '''Now that we have the observations, we need to loop through each of the testing files and permutate them'''
    
    save_permutation_figures = f'Outputs/permutation_data{region_name}'
    os.system(f'mkdir -p {save_permutation_figures}')

    #Get the files from the correct location (we need to loop through each of the models.
    # exps = sorted(glob(f'checkpoints/{region_name}/Wk{lead}/*'))
    exps = sorted(glob(f'Data/model_npy_inputs/{region_name}/Wk{lead}_EX_input_data/*testing*'))

    exps = [i for i in exps if 'RZSM' in i]
    exps = [i for i in exps if 'XGBOOST' not in i]

    #Make a list of the experiment names. We only need to grab a single file since they all have the same data
    '''We are going to seperate by hybrid and obs.driven. So remove EX0 and EX13'''
    EX_list = [f'EX{i}' for i in range(26)]
    EX_list = EX_list+['EX27']+['EX28']
    EX_list = [i for i in EX_list if 'EX0' not in i ]
    EX_list = [i for i in EX_list if 'EX13' not in i ]
    # EX_list = [i for i in EX_list if 'EX10' not in i ]

    return(obs_final_train,obs_final_validation,obs_final_testing,EX_list)

In [25]:
def return_the_experiment_input(exps,file,lead):
    '''First find the experiment name within the file'''
    ex_num = file.split(f'Wk{lead}_')[-1].split('_')[0]

    '''Now loop through exps to grab the correct input'''
    correct_file = [i for i in exps if ex_num in i]
    correct_file = [i for i in correct_file if 'XGBOOST' not in i]

    return(np.load(correct_file[0]),ex_num)

In [26]:
def permutation_test_by_lead(lead, test_year):
    #test 
    # lead = 1
    # test_year=2019

    day_num = (lead*7) -1
    
    file_path = f'Outputs/permutation_tests/mae_rmse_results'
    file_rmse_save = f'{file_path}/Wk{lead}_rmse_vals.pkl'
    file_mae_save = f'{file_path}/Wk{lead}_mae_vals.pkl'
    complete_rmse = f'{file_path}/Wk{lead}_rmse_complete.pkl'
    complete_mae = f'{file_path}/Wk{lead}_mae_complete.pkl'
    
    os.system(f'mkdir -p {file_path}')

    rmse_output, mae_output, rmse_complete, mae_complete = putils.return_rmse_and_mae_pickle_files(file_rmse_save, file_mae_save, complete_mae, complete_rmse)

    ###########################################################################

    leads = [6,13,20,27,34]

    obs_final_train,obs_final_validation,obs_final_testing,EX_list = return_data_and_experiment_numbers(region_name, leads,train_start, train_end, val_start, val_end, test_start, test_end,day_num)

    obs_final_testing_arr = obs_final_testing[putils.xarray_varname(obs_final_testing)].to_numpy()
    '''These are the actual experiment names below'''
    BC,OBS,HYB = verifications.return_experiment_colors_and_names()

    '''Loop through each model checkpoint'''
    checkpt_dir = f'checkpoints/{region_name}/Wk{lead}'
    file_list = sorted(glob(f'{checkpt_dir}/*regular*'))
    file_list = [i for i in file_list if '.' not in i] 
    file_list = [i for i in file_list if '/checkpoint' not in i]
    file_list = [i for i in file_list if 'ECMWF' not in i] #don't do ECMWF. We haven't coded it with different varabiles yet
    file_list = [i for i in file_list if BC[0] not in i]
    file_list = [i for i in file_list if BC[1] not in i]
    # file_list = [i for i in file_list if 'EX10' not in i ]
    # file_list = [i for i in file_list if 'EX14' not in i ]
    # file_list = [i for i in file_list if 'EX11' not in i ]
    # file_list = [i for i in file_list if 'EX12' not in i ]
    # file_list = [i for i in file_list if 'EX15' not in i ]
    file_list = [i for i in file_list if 'EX16' not in i ]
    # file_list = [i for i in file_list if 'EX17' not in i ]
    # file_list = [i for i in file_list if 'EX18' not in i ]
    if lead ==4:
        file_list = [i for i in file_list if 'EX12' not in i ]
        file_list = [i for i in file_list if 'EX21' not in i ]
        file_list = [i for i in file_list if 'EX28' not in i ]

    ##########################################################################
    if test_year == 2019:
        file_list = [i for i in file_list if '2012' not in i]
    else:
        file_list = [i for i in file_list if str(pd.to_datetime(test_end).year) in i]

    '''Experiment saved inputs'''
    exps = sorted(glob(f'Data/model_npy_inputs/{region_name}/Wk{lead}_EX_input_data/*testing*'))
    exps = [i for i in exps if 'XGBOOST' not in i]
    exps = [i for i in exps if 'mean' not in i]

    print(f'Working on file list {file_list}')

    #For masking
    RZSM_train_obs, RZSM_validation_obs = pred.return_masking_objects_for_RZSM(input_directory,final_testing_year)
    mask_zero = RZSM_validation_obs.squeeze()
    
    for file in file_list:
        print(f'Working on file experiment {file}')
        # break
        try:
            testing_input,ex_num = return_the_experiment_input(exps,file,lead)
            channel_list = f.load_channel_list_permutation(ex_num, lead)
            dont_continue=False
            
        except IndexError:
            dont_continue = True

        if dont_continue:
            pass
        else:
            if (len(channel_list) == testing_input.shape[-1]):
                model = load_model(file,compile=False) 
                model_name = file.split('/')[-1].split('_testing')[0]
                
                #These will contain the average MAE and RMSE across CONUS to plot
                save_for_plot_mae = {}
                save_for_plot_rmse = {}

                continue_ = False
                
                for idx,channel in enumerate(channel_list):
                    # break
                    '''Check if we have already completed it'''
                    unit_test = putils.check_if_already_completed_permuatation(rmse_complete, mae_complete, ex_num,  OBS, HYB, channel, model_name)

                    if unit_test == 'Not-Completed':
                        # break
                        print(f'\nPermutating channel {channel}\n')
    
                        new_input_with_noise,reforecast_nan,var_noise_min,var_noise_max,var_ = f.load_min_max_files_and_rescale_data(testing_input,channel,idx,file,region_name,day_num,test_year,lead)
    
                        try:
                            prediction_ = np.array(model.predict(new_input_with_noise))
                            prediction_.shape
                            
                            '''Just choose the very last prediction made'''
                            prediction_ = prediction_[-1,:,:,:,0]
        
                            yhat = verifications.reverse_min_max_scaling_for_permutations(prediction_,region_name,day_num,'GEFSv12',test_year,'soilw_bgrnd')
            
                            yhat = np.where(np.isnan(reforecast_nan),np.nan,yhat)
                            yhat = np.where(mask_zero == 0, np.nan, yhat)
                            # yhat = np.where(np.isnan(reforecast_nan),np.nan,yhat.squeeze())
                            yhat = np.reshape(yhat,(yhat.shape[0]//11,11,yhat.shape[1],yhat.shape[2]))
                
                            RZSM_mae =np.nanmean(np.abs(obs_final_testing_arr -  yhat),axis=(0,1))
                
                            RZSM_rmse = np.nanmean((obs_final_testing_arr -  yhat)**2,axis=(0,1))
    
                            save_for_plot_rmse[channel] = np.nanmean(RZSM_rmse)
                            save_for_plot_mae[channel] = np.nanmean(RZSM_mae)
                        
                            '''Now add to the dictionary for RMSE'''
                            if ex_num in OBS:
                                try:
                                    # if output['OBS'][channel]['Value']
                                    rmse_output['OBS'][channel]['Value'] = rmse_output['OBS'][channel]['Value'] + RZSM_rmse
                                    rmse_output['OBS'][channel]['Num_experiments'] = rmse_output['OBS'][channel]['Num_experiments'] + 1
                                    rmse_complete['OBS'][model_name].append(channel)
                                except KeyError:
                                    rmse_output['OBS'][channel] = {'Num_experiments':1, 'Value':RZSM_rmse}
                                    rmse_complete = putils.append_to_complete_list(rmse_complete, model_name, channel, 'OBS')
                                    
                            elif ex_num in HYB:
                                try:
                                    rmse_output['HYB'][channel]['Value'] = rmse_output['HYB'][channel]['Value'] + RZSM_rmse
                                    rmse_output['HYB'][channel]['Num_experiments'] = rmse_output['HYB'][channel]['Num_experiments'] + 1
                                    rmse_complete['HYB'][model_name].append(channel)
                                except KeyError:
                                    rmse_output['HYB'][channel] = {'Num_experiments':1, 'Value':RZSM_rmse}
                                    rmse_complete = putils.append_to_complete_list(rmse_complete, model_name, channel, 'HYB')

                                    
                            '''Now add to the dictionary for MAE only'''
                            if ex_num in OBS:
                                try:
                                    mae_output['OBS'][channel]['Value'] = mae_output['OBS'][channel]['Value'] + RZSM_mae
                                    mae_output['OBS'][channel]['Num_experiments'] = mae_output['OBS'][channel]['Num_experiments'] + 1
                                    mae_complete['OBS'][model_name].append(channel)
                                except KeyError:
                                    mae_output['OBS'][channel] = {'Num_experiments':1, 'Value':RZSM_mae}
                                    mae_complete = putils.append_to_complete_list(mae_complete, model_name, channel, 'OBS')

                            elif ex_num in HYB:
                                try:
                                    mae_output['HYB'][channel]['Value'] = mae_output['HYB'][channel]['Value'] + RZSM_mae
                                    mae_output['HYB'][channel]['Num_experiments'] = mae_output['HYB'][channel]['Num_experiments'] + 1
                                    mae_complete['HYB'][model_name].append(channel)
                                except KeyError:
                                    mae_output['HYB'][channel] = {'Num_experiments':1, 'Value':RZSM_mae}
                                    mae_complete = putils.append_to_complete_list(mae_complete, model_name, channel, 'HYB')

    
                            continue_ = True
                        
                        except ValueError:
                            continue_ = False
                            pass
                        
                if continue_:
                    plot_barplot_mae_rmse(save_for_plot_mae, save_for_plot_rmse, lead, model_name)
                    
            
        '''Saves after each file is completed'''
        with open(file_rmse_save, 'wb') as file:
            pickle.dump(rmse_output, file)
    
        with open(file_mae_save, 'wb') as file:
            pickle.dump(mae_output, file)
        
        with open(complete_rmse, 'wb') as file:
            pickle.dump(rmse_complete, file)
    
        with open(complete_mae, 'wb') as file:
            pickle.dump(mae_complete, file)

    return(f'Completed week {lead}')

In [27]:
# def permutation_test_by_lead_by_experiment(lead, test_year, experiment_number):
#     #test 
#     # lead = 1
#     # test_year=2019
    
#     day_num = (lead*7) -1

#     try:
#         with open(file_rmse_save, "rb") as f:
#             rmse_output = pickle.load(f)
    
#         with open(file_mae_save, "rb") as f:
#             mae_output = pickle.load(f)    
#     except FileNotFoundError:
#         rmse_output = {}
#         rmse_output['OBS'] = {}
#         rmse_output['HYB'] = {}
    
#         mae_output = {}
#         mae_output['OBS'] = {}
#         mae_output['HYB'] = {}

    
#     leads = [6,13,20,27,34]

#     obs_final_train,obs_final_validation,obs_final_testing,EX_list = return_data_and_experiment_numbers(region_name, leads,train_start, train_end, val_start, val_end, test_start, test_end,day_num)

#     obs_final_testing_arr = obs_final_testing[putils.xarray_varname(obs_final_testing)].to_numpy()
#     '''These are the actual experiment names below'''
#     BC,OBS,HYB = verifications.return_experiment_colors_and_names()


#     '''Loop through each model checkpoint'''
#     checkpt_dir = f'checkpoints/{region_name}/Wk{lead}'
#     file_list = sorted(glob(f'{checkpt_dir}/*{experiment_number}_regular*'))
  
    
#     if test_year == 2019:
#         file_list = [i for i in file_list if '2012' not in i]
#     else:
#         file_list = [i for i in file_list if str(pd.to_datetime(test_end).year) in i]

#     '''Experiment saved inputs'''
#     exps = sorted(glob(f'Data/model_npy_inputs/{region_name}/Wk{lead}_EX_input_data/*testing*'))
#     exps = [i for i in exps if 'XGBOOST' not in i]
#     exps = [i for i in exps if 'mean' not in i]

#     print(f'Working on file list {file_list}')

#     #For masking
#     RZSM_train_obs, RZSM_validation_obs = pred.return_masking_objects_for_RZSM(input_directory,final_testing_year)
#     mask_zero = RZSM_validation_obs.squeeze()
    
#     for file in file_list:
#         print(f'Working on file experiment {file}')
#         try:
#             testing_input,ex_num = return_the_experiment_input(exps,file,lead)
#             channel_list = f.load_channel_list_permutation(ex_num, lead)
#             dont_continue=False
            
#         except IndexError:
#             dont_continue = True

#         if dont_continue:
#             pass
#         else:
#             if len(channel_list) == testing_input.shape[-1]:
#                 model = load_model(file,compile=False) 
#                 model_name = file.split('/')[-1].split('_testing')[0]
                
#                 #These will contain the average MAE and RMSE across CONUS to plot
#                 save_for_plot_mae = {}
#                 save_for_plot_rmse = {}
                
#                 for idx,channel in enumerate(channel_list):
#                     '''Now add to the dictionary'''
#                     if ex_num in OBS:
#                         try:
#                             rmse_output['OBS'][channel]['Value'] = rmse_output['OBS'][channel]['Value'] + RZSM_rmse
#                             rmse_output['OBS'][channel]['Num_experiments'] = rmse_output['OBS'][channel]['Num_experiments'] + 1
#                         except KeyError:
#                             rmse_output['OBS'][channel] = {'Num_experiments':1, 'Value':RZSM_rmse}
#                     elif ex_num in HYB:
                        
                
#                     # idx,channel = 6, 'pwat_obs_lag-1'
#                     # idx, channel = 0, 'RZSM_obs_lag-1'
                    
#                     new_input_with_noise,reforecast_nan,var_noise_min,var_noise_max,var_ = f.load_min_max_files_and_rescale_data(testing_input,channel,idx,file,region_name,day_num,test_year,lead)

#                     try:
#                         prediction_ = np.array(model.predict(new_input_with_noise))
#                         prediction_.shape
                        
#                         '''Just choose the very last prediction made'''
#                         prediction_ = prediction_[-1,:,:,:,0]
        
#                         yhat = verifications.reverse_min_max_scaling_for_permutations(prediction_,region_name,day_num,'GEFSv12',test_year,'soilw_bgrnd')
        
#                         yhat = np.where(np.isnan(reforecast_nan),np.nan,yhat)
#                         yhat = np.where(mask_zero == 0, np.nan, yhat)
#                         # yhat = np.where(np.isnan(reforecast_nan),np.nan,yhat.squeeze())
#                         yhat = np.reshape(yhat,(yhat.shape[0]//11,11,yhat.shape[1],yhat.shape[2]))
            
#                         RZSM_mae =np.nanmean(np.abs(obs_final_testing_arr -  yhat),axis=(0,1))
            
#                         RZSM_rmse = np.nanmean((obs_final_testing_arr -  yhat)**2,axis=(0,1))
    
#                         save_for_plot_rmse[channel] = np.nanmean(RZSM_rmse)
#                         save_for_plot_mae[channel] = np.nanmean(RZSM_mae)
                        
#                         '''Now add to the dictionary'''
#                         if ex_num in OBS:
#                             try:
#                                 rmse_output['OBS'][channel]['Value'] = rmse_output['OBS'][channel]['Value'] + RZSM_rmse
#                                 rmse_output['OBS'][channel]['Num_experiments'] = rmse_output['OBS'][channel]['Num_experiments'] + 1
#                             except KeyError:
#                                 rmse_output['OBS'][channel] = {'Num_experiments':1, 'Value':RZSM_rmse}
#                         elif ex_num in HYB:
#                             try:
#                                 rmse_output['HYB'][channel]['Value'] = rmse_output['HYB'][channel]['Value'] + RZSM_rmse
#                                 rmse_output['HYB'][channel]['Num_experiments'] = rmse_output['HYB'][channel]['Num_experiments'] + 1
#                             except KeyError:
#                                 rmse_output['HYB'][channel] = {'Num_experiments':1, 'Value':RZSM_rmse}
    
    
#                         '''Now add to the dictionary'''
#                         if ex_num in HYB:
#                             try:
#                                 mae_output['OBS'][channel]['Value'] = mae_output['OBS'][channel]['Value'] + RZSM_mae
#                                 mae_output['OBS'][channel]['Num_experiments'] = mae_output['OBS'][channel]['Num_experiments'] + 1
#                             except KeyError:
#                                 mae_output['OBS'][channel] = {'Num_experiments':1, 'Value':RZSM_mae}
#                         elif ex_num in HYB:
#                             try:
#                                 mae_output['HYB'][channel]['Value'] = mae_output['HYB'][channel]['Value'] + RZSM_mae
#                                 mae_output['HYB'][channel]['Num_experiments'] = mae_output['HYB'][channel]['Num_experiments'] + 1
#                             except KeyError:
#                                 mae_output['HYB'][channel] = {'Num_experiments':1, 'Value':RZSM_mae}

#                         continue_ = True
                    
            
                    
#                     except ValueError:
#                         continue_ = False
#                         pass
                        
#                 if continue_:
#                     plot_barplot_mae_rmse(save_for_plot_mae, save_for_plot_rmse, lead, model_name)
                    

#     return(f'Completed week {lead}')

In [28]:
def run_model_and_prediction(experiment):
    global lead_week
    for lead_week in [1]:
        global lead
        lead=lead_week
        day_lead = (lead*7)-1
    
        generate_model_inputs(region_name=experiment['region_name'], lead = lead_week, num_lags_obs_RZSM=experiment['num_lags_obs_RZSM'],
                              include_lags_obs_pwat_spfh_tmax=experiment['include_lags_obs_pwat_spfh_tmax'],include_reforecast_or_not=experiment['include_reforecast_or_not'], 
                              addtl_experiment = experiment['addtl_experiment'], experiment_test = experiment['experiment_test'])
    
        run_EXPERIMENT(region_name=experiment['region_name'], lead = lead_week, num_lags_obs_RZSM=experiment['num_lags_obs_RZSM'],
                              include_lags_obs_pwat_spfh_tmax=experiment['include_lags_obs_pwat_spfh_tmax'],include_reforecast_or_not=experiment['include_reforecast_or_not'], 
                              addtl_experiment = experiment['addtl_experiment'], experiment_test = experiment['experiment_test'])

        def open_min_max_GEFS(train_start, train_end, val_start, val_end, test_start, test_end,day_lead):
            print('Loading GEFSv12 min max files')
            file = xr.open_mfdataset('Data/GEFSv12_reforecast/soilw_bgrnd/min_max_RZSM/*')
            train = file.sel(S=slice(train_start,train_end)).sel(L=day_lead).stack(combine_models=['S','M']).transpose('combine_models','Y','X').to_array().squeeze().values
            val = file.sel(S=slice(val_start,val_end)).sel(L=day_lead).stack(combine_models=['S','M']).transpose('combine_models','Y','X').to_array().squeeze().values
            test = file.sel(S=slice(test_start,test_end)).sel(L=day_lead).stack(combine_models=['S','M']).transpose('combine_models','Y','X').to_array().squeeze().values
            return(train,test,val)

        train,test,val = open_min_max_GEFS(train_start, train_end, val_start, val_end, test_start, test_end,day_lead)
            
    
        make_single_predictions_training_validation(region_name=experiment['region_name'], lead = lead_week, num_lags_obs_RZSM=experiment['num_lags_obs_RZSM'],
                              include_lags_obs_pwat_spfh_tmax=experiment['include_lags_obs_pwat_spfh_tmax'],include_reforecast_or_not=experiment['include_reforecast_or_not'], 
                              addtl_experiment = experiment['addtl_experiment'], experiment_test = experiment['experiment_test'], train=train, val = val)

        make_single_predictions_testing(region_name=experiment['region_name'], lead = lead_week, num_lags_obs_RZSM=experiment['num_lags_obs_RZSM'],
                              include_lags_obs_pwat_spfh_tmax=experiment['include_lags_obs_pwat_spfh_tmax'],include_reforecast_or_not=experiment['include_reforecast_or_not'], 
                              addtl_experiment = experiment['addtl_experiment'], experiment_test = experiment['experiment_test'], test = test)
        
    print(f'Completed {experiment}')

In [29]:
EX9 ={'region_name':region_name, 'num_lags_obs_RZSM': 3, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':0 }
experiment=EX9
#for testing
lead,lead_week =1,1

experiment['region_name']
lead = lead_week
num_lags_obs_RZSM=experiment['num_lags_obs_RZSM']
include_lags_obs_pwat_spfh_tmax=experiment['include_lags_obs_pwat_spfh_tmax']
include_reforecast_or_not=experiment['include_reforecast_or_not']
addtl_experiment = experiment['addtl_experiment']
experiment_test = experiment['experiment_test']

epochs = 150
batch_size = 66
initial_learning_rate=0.0001
beta_1 = 0.9
shuffle=False
patience=10
kernel_norm = None
deep_supervision = True
number_of_UNET_backbone_max_pool=4


In [30]:
run_model_and_prediction(experiment)
















tt


Now loading Verification Data from Observations.

Loading previously created data



KeyboardInterrupt



In [ ]:

run_model_and_prediction(experiment)


In [ ]:
all_exps= [EX0,EX1,EX2,EX3,EX4,EX5,EX6,EX7,EX8,EX9,EX10,EX11,EX13,EX14,EX15,EX16,EX17,EX18,EX19,EX20,EX21,EX22,EX23,EX24,EX25,EX27,EX28]

'''We are not testing EX12 or EX26'''
for idx,experiment in enumerate(all_exps):
    run_model_and_prediction(experiment)

In [ ]:
'''Run the single prediction tests'''
        
%load_ext autoreload 
%autoreload 2
%reload_ext autoreload
    
testing_scenarios = ['dense', 'regular', 'transformer', 'super_pixel', 'attention', 'denseLarge', 'basic']

for testing_scenario in testing_scenarios:
    #removed EX12 and EX26
    all_exps= [EX0,EX1,EX2,EX3,EX4,EX5,EX6,EX7,EX8,EX9,EX10,EX11,EX13,EX14,EX15,EX16,EX17,EX18,EX19,EX20,EX21,EX22,EX23,EX24,EX25,EX27,EX28]

    
    global save_loss_name, loss_fn
    
    if testing_scenario == 'dense':
        import modelRzsmDenseRelu as UNETRzsm
        
        save_loss_name = 'denseLoss'
        loss_fn = losses.crps2d_tf_dense_test2
        
    elif testing_scenario == 'regular':
        import modelRzsmReluExtraConv as UNETRzsm
    
        save_loss_name = 'regular'
        loss_fn = losses.crps2d_tf
    
    elif testing_scenario == 'transformer':
        import modelRzsmReluTransformer as UNETRzsm
        
        save_loss_name = 'transformer'
        loss_fn = losses.crps2d_tf_dense_test2
    
    elif testing_scenario == 'super_pixel':
        import modelRzsmReluPixelSuper as UNETRzsm
        
        save_loss_name = 'supPixel'
        loss_fn = losses.crps2d_tf_dense_test2
    
    elif testing_scenario == 'attention':
        import modelRzsmReluAttention as UNETRzsm
        
        save_loss_name = 'attention'
        loss_fn = losses.crps2d_tf_dense_test2
        
    elif testing_scenario == 'None':
        import modelRzsm2DdropoutRelu as UNETRzsm
        
        save_loss_name = ''
        loss_fn = losses.crps2d_tf
    
    elif testing_scenario == 'denseLarge':
        import modelRzsmReluExtraConv as UNETRzsm
    
        save_loss_name = 'denseLarge'
        loss_fn = losses.crps2d_tf_dense_test2
        
    elif testing_scenario == 'basic':
        import basicUNET as UNETRzsm
    
        save_loss_name = testing_scenario
        loss_fn = losses.crps2d_tf

    for idx,experiment in enumerate(all_exps):
        run_model_and_prediction(experiment)

In [ ]:
all_exps= [EX0,EX1,EX2,EX3,EX4,EX5,EX6,EX7,EX8,EX9,EX10,EX11,EX12,EX13,EX14,EX15,EX16,EX17,EX18,EX19,EX20,EX21,EX22,EX23,EX24,EX25,EX27,EX28]
for idx,experiment in enumerate(all_exps):
    run_model_and_prediction(experiment)

In [ ]:
for idx,experiment in enumerate(all_exps):
    run_model_and_prediction(experiment)

# After all the experiments are completed, now do a permutation test
## Save the results for each type (observation or hybrid driven) only

In [ ]:
for lead in [1,2]:
    permutation_test_by_lead(lead=lead, test_year=2019)

In [ ]:
for lead in [1]:
    permutation_test_by_lead_by_experiment(lead=lead, test_year=2019, experiment_number = 'EX1')